# C++ Cling Examples for STM32F4Discovery

![Image](img/head.png)

Load [REMCU](https://remotemcu.github.io) shared libray

In [ ]:
.L libremcu

Add path with header files

In [ ]:
.I remcu_include/

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [ ]:
#include "remcu.h"
#include "stm32f4xx.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.github.io/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.github.io/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)

Before that, plug stm32 board with debugger to PC

run openocd utility in command line:
```
openocd -f board/stm32f4discovery.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative
bool connection = false;

In [ ]:
connection = remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//connection = remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

In [ ]:
if(connection == false){
    fprintf(stderr, "1. Check connection to debug server\n");
    fprintf(stderr, "2. Check debug server running\n");
    fprintf(stderr, "3. Check the debugged MCU is correct\n");
    exit(1);
}

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):

In [ ]:
remcu_resetRemoteUnit(__HALT)

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (startup_stm32f4xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f4xx.c file

In [ ]:
SystemInit();

### GPIO Example
![Image](img/gpio.png)
>The code below have been taken from **IO_Toggle** ([main.c](../STM32F4-Discovery_FW_V1.1.0/Project/Peripheral_Examples/IO_Toggle/main.c)) example of [Standard Peripheral Library(SPL) for STM32F4DISCOVERY](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-library-expansion/stsw-stm32068.html#overview)

Clock enable and configure pins:

In [ ]:
/* GPIOD Periph clock enable */
RCC_AHB1PeriphClockCmd(RCC_AHB1Periph_GPIOD, ENABLE);
GPIO_InitTypeDef  GPIO_InitStructure;

/* Configure PD12, PD13, PD14 and PD15 in output pushpull mode */
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_12 | GPIO_Pin_13| GPIO_Pin_14| GPIO_Pin_15;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_OUT;
GPIO_InitStructure.GPIO_OType = GPIO_OType_PP;
GPIO_InitStructure.GPIO_Speed = GPIO_Speed_100MHz;
GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL;
GPIO_Init(GPIOD, &GPIO_InitStructure);

Flash leds on:

In [ ]:
/* PD12 to be toggled */
    GPIO_SetBits(GPIOD, GPIO_Pin_12);
/* PD13 to be toggled */
    GPIO_SetBits(GPIOD, GPIO_Pin_13);
/* PD14 to be toggled */
    GPIO_SetBits(GPIOD, GPIO_Pin_14);
/* PD15 to be toggled */
    GPIO_SetBits(GPIOD, GPIO_Pin_15);

<details>
  <summary><b>Result(click to show)</b></summary>

![Image](img/led_on.png)
</details>

Flash leds off:

In [ ]:
GPIO_ResetBits(GPIOD, GPIO_Pin_12|GPIO_Pin_13|GPIO_Pin_14|GPIO_Pin_15);

If it didn’t happen, you can use the utility functions to detect errors...

### Utility Functions

The ***remcu_getErrorCount***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function returns the number of errors that occurred after connection:

In [ ]:
remcu_getErrorCount()

If no error occurred, result ***0***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

The ***remcu_isConnected***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function indicates status of the current debug server connection:

In [ ]:
remcu_isConnected()

Result: connected ***true*** , disconnected ***false***

Some of debug adapters can have hardware problems. The ***remcu_debuggerTest*** (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function enables to detect the such adapters. The function returns a null pointer if the test is successful:

In [ ]:
remcu_debuggerTest()

If no error occurred, result ***nullptr***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

>If bugs are not discovered and the chip don't responds to driver function call or the result of work was not what you expected, see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)


### PWM example
![Image](img/pwm_wave.png)
This example shows how to configure the TIM peripheral in PWM (Pulse Width Modulation) mode.

>The code below have been based on **TIM_PWM_Output** ([main.c](../STM32F4-Discovery_FW_V1.1.0/Project/Peripheral_Examples/TIM_PWM_Output/main.c)) example of [Standard Peripheral Library(SPL) for STM32F4DISCOVERY](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-library-expansion/stsw-stm32068.html#overview)

GPIOC Configuration: TIM4 CH1 (PD12) and TIM4 CH2 (PD13)

In [ ]:
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_12 | GPIO_Pin_13 ;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_AF;
GPIO_InitStructure.GPIO_Speed = GPIO_Speed_100MHz;
GPIO_InitStructure.GPIO_OType = GPIO_OType_PP;
GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_UP ;
GPIO_Init(GPIOD, &GPIO_InitStructure); 
/* Connect TIM4 pins to AF2 */  
GPIO_PinAFConfig(GPIOD, GPIO_PinSource12, GPIO_AF_TIM4);
GPIO_PinAFConfig(GPIOD, GPIO_PinSource13, GPIO_AF_TIM4);

 TIM4 Configuration: generate 2 PWM signals with 2 different duty cycles.
 
    In this example TIM4 input clock (TIM4CLK) is set to 1/0xFFFF * APB1 clock (PCLK1), 
    since APB1 prescaler is different from 1.   
      TIM4CLK = PCLK1 / 0xFFFF
      PCLK1 = HCLK / 4 
      => TIM4CLK = HCLK / 2 = SystemCoreClock /2
    Note: 
     SystemCoreClock variable holds HCLK frequency and is defined in system_stm32f4xx.c file.
     Each time the core clock (HCLK) changes, user had to call SystemCoreClockUpdate()
     function to update SystemCoreClock variable value. Otherwise, any configuration
     based on this variable will be incorrect.  
     
    The TIM4 CCR1 register value is equal to 0xFFFF/2, so the TIM4 Channel 1 generates a 
    PWM signal with a frequency equal to ~1 KHz and a duty cycle equal to 50%:
    TIM4 Channel1 duty cycle = (TIM4_CCR1/ TIM4_ARR + 1)* 100 = 50%

    The TIM4 CCR2 register value is equal to 0xFFFF/4, so the TIM4 Channel 2 generates a 
    PWM signal with a frequency equal to ~1 KHz and a duty cycle equal to 25%:
    TIM4 Channel2 duty cycle = (TIM4_CCR2/ TIM4_ARR + 1)* 100 = 25%
    
    The PWM waveform can be displayed using an oscilloscope.
    
    @par Hardware and Software environment 

  - This example runs on STM32F4xx Devices Revision A.
  
  - This example has been tested with STM32F4-Discovery (MB997) RevA and can be
    easily tailored to any other development board.    

In [ ]:
// TIM4 Periph clock enable:
RCC_APB1PeriphClockCmd(RCC_APB1Periph_TIM4, ENABLE);

TIM_TimeBaseInitTypeDef  TIM_TimeBaseStructure;
TIM_OCInitTypeDef  TIM_OCInitStructure;

/* Time base configuration */
TIM_TimeBaseStructure.TIM_Period = 0xFFFF;
TIM_TimeBaseStructure.TIM_Prescaler = 0xFFFF;
TIM_TimeBaseStructure.TIM_ClockDivision = 0xFFFF;
TIM_TimeBaseStructure.TIM_CounterMode = TIM_CounterMode_Up;

TIM_TimeBaseInit(TIM4, &TIM_TimeBaseStructure);

/* PWM1 Mode configuration: Channel1 */
TIM_OCInitStructure.TIM_OCMode = TIM_OCMode_PWM1;
TIM_OCInitStructure.TIM_OutputState = TIM_OutputState_Enable;
TIM_OCInitStructure.TIM_Pulse = 0xFFFF/2;
TIM_OCInitStructure.TIM_OCPolarity = TIM_OCPolarity_High;

TIM_OC1Init(TIM4, &TIM_OCInitStructure);

/* PWM1 Mode configuration: Channel2 */
TIM_OCInitStructure.TIM_OutputState = TIM_OutputState_Enable;
TIM_OCInitStructure.TIM_Pulse = 0xFFFF/4;

TIM_OC2Init(TIM4, &TIM_OCInitStructure);

/* TIM3 enable counter */
TIM_Cmd(TIM4, ENABLE);

Now you can control the duty cycle and adjust the brightness of the board LEDs.

Change the second argument of TIM_SetCompare(1,2) function:

In [ ]:
TIM_SetCompare1(TIM4, 0); //Set Duty Cycle of TIM4 channel 1
TIM_SetCompare2(TIM4, 0); //Set Duty Cycle of TIM4 channel 2

Create two slider elements to easily change  the brightness of the board LEDs.
> First install [**xwidgets**](https://github.com/jupyter-xeus/xwidgets) or make sure you have it
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [ ]:
#include "xwidgets/xslider.hpp"
xw::slider<int> sliderBrightness_Red;
xw::slider<int> sliderBrightness_Green;

sliderBrightness_Red.min = sliderBrightness_Green.min = 0;
sliderBrightness_Red.max = sliderBrightness_Green.max = 0xFFFF;

sliderBrightness_Red.display();
sliderBrightness_Green.display();

XOBSERVE(sliderBrightness_Red, value, [](const auto& Brightness) {
    TIM_SetCompare1(TIM4, Brightness.value);
});

XOBSERVE(sliderBrightness_Green, value, [](const auto& Brightness) {
    TIM_SetCompare2(TIM4, Brightness.value);
});

[video demo](img/cling_slider.mpg)

## ADC-DMA Example
![Image](img/adc_dma.png)
This example describes how to use the ADC3 and DMA to transfer continuously 
converted data from ADC3 to memory.

>The code below have been taken from **ADC3_DMA** ([main.c](../STM32F4-Discovery_FW_V1.1.0/Project/Peripheral_Examples/ADC3_DMA/main.c)) example of [Standard Peripheral Library(SPL) for STM32F4DISCOVERY](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-library-expansion/stsw-stm32068.html#overview)

The ADC3 is configured to convert continuously channel12. 
The DMA store value of ADC3 DR register to the MCU memory, in circular mode.
To read the ADC data, this script stops ADC3 then reads the memory and restarts work of ADC3.

In this example, the system clock is 144MHz, APB2 = 72MHz and ADC clock = APB2/2. 
Since ADC3 clock is 36 MHz and sampling time is set to 3 cycles, the conversion 
time to 12bit data is 12 cycles so the total conversion time is (12+3)/36= 0.41us(2.4Msps).

The embedded canvas below will plot ADC value or voltage vs time chart in the interval 52us:  
ADC_BUFF_SIZE x (total conversion time) = 32x4 x (0.41us) = 52,48us  
<details>
  <summary>Canvas <b>(click to show)</b></summary>

![Image](img/adc_chart.png)
</details>

@par Hardware and Software environment 

  - This example runs on STM32F4xx Devices Revision A.
  
  - This example has been tested with STM32F4-Discovery (MB997) RevA and can be
    easily tailored to any other development board.
    
  - STM32F4-Discovery 
    - Connect the external signal(ranges from 0 to 3.3V) to the ADC3 pin (PC.02) to be converted.
    

**ptr_remote_RAM** is a starting address of the RAM region. For STM32F4, it is 0x20000000 (from [datasheet](https://www.st.com/resource/en/datasheet/stm32f405rg.pdf))
![Image](img/starting_address.png)


>Note that Community and Education versions have restrictions on memory region where you store and load data. You can use first 8 KBytes of first memory bank(STM32F407 chip). It is from 0x20000000 to (0x20000000 + 8*1024) address. To clarify that, see the [Download](https://remotemcu.github.io/download) page of your chip or the START_AVAILABLE_MEMORY_REGION and EMD_AVAILABLE_MEMORY_REGION constants in device_defines.h(is in remcu_include folder)

In [ ]:
const uint32_t ptr_remote_RAM = 0x20000000;
const size_t ADC_BUFF_SIZE = 32*4;

ADC3 configuration 
- Enable peripheral clocks
- DMA2_Stream0 channel2 configuration
- Configure ADC Channel12 pin as analog input
- Configure ADC3 Channel12

In [ ]:
  ADC_InitTypeDef       ADC_InitStructure;
  ADC_CommonInitTypeDef ADC_CommonInitStructure;
  DMA_InitTypeDef       DMA_InitStructure;

#define ADC3_DR_ADDRESS     ((uint32_t)0x4001224C)

  /* Enable ADC3, DMA2 and GPIO clocks ****************************************/
  RCC_AHB1PeriphClockCmd(RCC_AHB1Periph_DMA2 | RCC_AHB1Periph_GPIOC, ENABLE);
  RCC_APB2PeriphClockCmd(RCC_APB2Periph_ADC3, ENABLE);

  /* DMA2 Stream0 channel0 configuration **************************************/
  DMA_InitStructure.DMA_Channel = DMA_Channel_2;  
  DMA_InitStructure.DMA_PeripheralBaseAddr = (uint32_t)ADC3_DR_ADDRESS;
  DMA_InitStructure.DMA_Memory0BaseAddr = ptr_remote_RAM;
  DMA_InitStructure.DMA_DIR = DMA_DIR_PeripheralToMemory;
  DMA_InitStructure.DMA_BufferSize = ADC_BUFF_SIZE; //32*4
  DMA_InitStructure.DMA_PeripheralInc = DMA_PeripheralInc_Disable;
  //DMA_InitStructure.DMA_MemoryInc = DMA_MemoryInc_Disable;
  DMA_InitStructure.DMA_MemoryInc = DMA_MemoryInc_Enable;
  DMA_InitStructure.DMA_PeripheralDataSize = DMA_PeripheralDataSize_HalfWord;
  DMA_InitStructure.DMA_MemoryDataSize = DMA_MemoryDataSize_HalfWord;
  DMA_InitStructure.DMA_Mode = DMA_Mode_Circular;
  DMA_InitStructure.DMA_Priority = DMA_Priority_High;
  DMA_InitStructure.DMA_FIFOMode = DMA_FIFOMode_Disable;         
  DMA_InitStructure.DMA_FIFOThreshold = DMA_FIFOThreshold_HalfFull;
  DMA_InitStructure.DMA_MemoryBurst = DMA_MemoryBurst_Single;
  DMA_InitStructure.DMA_PeripheralBurst = DMA_PeripheralBurst_Single;
  DMA_Init(DMA2_Stream0, &DMA_InitStructure);
  DMA_Cmd(DMA2_Stream0, ENABLE);

  /* Configure ADC3 Channel12 pin as analog input ******************************/
  GPIO_InitStructure.GPIO_Pin = GPIO_Pin_2;
  GPIO_InitStructure.GPIO_Mode = GPIO_Mode_AN;
  GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL ;
  GPIO_Init(GPIOC, &GPIO_InitStructure);

  /* ADC Common Init **********************************************************/
  ADC_CommonInitStructure.ADC_Mode = ADC_Mode_Independent;
  ADC_CommonInitStructure.ADC_Prescaler = ADC_Prescaler_Div2;
  ADC_CommonInitStructure.ADC_DMAAccessMode = ADC_DMAAccessMode_Disabled;
  ADC_CommonInitStructure.ADC_TwoSamplingDelay = ADC_TwoSamplingDelay_5Cycles;
  ADC_CommonInit(&ADC_CommonInitStructure);

  /* ADC3 Init ****************************************************************/
  ADC_InitStructure.ADC_Resolution = ADC_Resolution_12b;
  ADC_InitStructure.ADC_ScanConvMode = DISABLE;
  ADC_InitStructure.ADC_ContinuousConvMode = ENABLE;
  ADC_InitStructure.ADC_ExternalTrigConvEdge = ADC_ExternalTrigConvEdge_None;
  ADC_InitStructure.ADC_DataAlign = ADC_DataAlign_Right;
  ADC_InitStructure.ADC_NbrOfConversion = 1;
  ADC_Init(ADC3, &ADC_InitStructure);

  /* ADC3 regular channel12 configuration *************************************/
  ADC_RegularChannelConfig(ADC3, ADC_Channel_12, 1, ADC_SampleTime_3Cycles);

 /* Enable DMA request after last transfer (Single-ADC mode) */
  ADC_DMARequestAfterLastTransferCmd(ADC3, ENABLE);

  /* Enable ADC3 DMA */
  ADC_DMACmd(ADC3, ENABLE);

  /* Enable ADC3 */
  ADC_Cmd(ADC3, ENABLE);

Start ADC3 Software Conversion

In [ ]:
ADC_SoftwareStartConv(ADC3);

Test ***remcu_loadFrMem***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function:

In [ ]:
uint16_t buff[16];
remcu_loadFrMem(ptr_remote_RAM, 32, (uint8_t*)buff)

In [ ]:
buff

Prepare canvas, which shows ADC data. [Template](https://github.com/QuantStack/xplot/blob/3399c4b96dffb1fec9d9c44de74d3d223bb04789/notebooks/xplot.ipynb)
> First install [xplot](https://github.com/QuantStack/xplot) or make sure you have it  
 
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [ ]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

xpl::figure fig;
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
std::vector<uint16_t> adc_data(ADC_BUFF_SIZE );
std::vector<int> x_line(ADC_BUFF_SIZE);
int i = 0;
for( auto & x : x_line){x = i++;}
line.x = x_line;
line.y = adc_data;
fig.add_mark(line);
    
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);
fig

Plug a signal generator into the ***PC2*** pin and run the code below. You will get a wave of the signal:

The graph above have been gotten for a sinus signal.
<details>
  <summary><b>Scheme (click to show)</b></summary>

![Image](img/adc_scheme.png)
</details>

>Note that Community and Education versions of the [REMCU](https://remotemcu.github.io/) library have a ***32-bytes limit*** on a memory operation. Therefore, if you need to copy big data, you can do it in parts.  
Also the Community and Education versions have restrictions on memory region where you store and load data. You can use first 8 KBytes of first memory bank(STM32F407 chip). It is from 0x20000000 to (0x20000000 + 8*1024) address. To clarify that, see the [Download](https://remotemcu.github.io/download) page of your chip or the START_AVAILABLE_MEMORY_REGION and EMD_AVAILABLE_MEMORY_REGION constants in device_defines.h(is in remcu_include folder)

In [ ]:
    uint16_t osc_buffer[ADC_BUFF_SIZE] = {0};


In [ ]:
#include <unistd.h>

for(int j=0; j< 30; j++){
    //Stop ADC3 module. ADC and DMA don't write to memory
    ADC_Cmd(ADC3, DISABLE);
    //Copy 32 blocks from MCU memory
    for(size_t i = 0; i < ADC_BUFF_SIZE*2; i += 32){ 
        remcu_loadFrMem(ptr_remote_RAM + i, 32, (uint8_t*)osc_buffer + i);
    }
    //The ADC’s DMA channel saves its data in a circular fashion 
    //and keeps on overwriting old data until is paused. 
    //left shift of ADC data for a picture like oscilloscope screen
    size_t shift = ADC_BUFF_SIZE - DMA_GetCurrDataCounter(DMA2_Stream0);
    for(size_t i = 0; i < ADC_BUFF_SIZE; i++){ 
        int shift_pos = (i + shift) % ADC_BUFF_SIZE;
        adc_data[i] = osc_buffer[shift_pos];
    }
    
    //plot ADC data
    line.y = adc_data;
    //Turn on ADC3
    ADC_Cmd(ADC3, ENABLE);
    //Continue ADC convertion and the DMA writes to memory region (ptr_remote_RAM)
    ADC_SoftwareStartConv(ADC3);
    usleep(200000);
}

<details>
  <summary>Result <b>(click to show)</b></summary>

![Image](img/adc_chart1.png)
</details>

### DAC - DMA Sine Wave Example
![Image](img/dac_dma.png)

This example provides a short description of how to use the DAC peripheral to 
generate Sine wave signals using DMA controller.

>The code below have been taken from **DAC_SignalsGeneration** ([main.c](../STM32F4-Discovery_FW_V1.1.0/Project/Peripheral_Examples/DAC_SignalsGeneration/main.c)) example of [Standard Peripheral Library(SPL) for STM32F4DISCOVERY](https://www.st.com/content/st_com/en/products/embedded-software/mcu-mpu-embedded-software/stm32-embedded-software/stm32-standard-peripheral-library-expansion/stsw-stm32068.html#overview)

@par Hardware and Software environment

  - This example runs on STM32F4xx Devices Revision A.
  
  - This example has been tested with STM32F4-Discovery (MB997) RevA and can be
    easily tailored to any other development board.

  - STM32F4-Discovery 	
     - Use USER push-button connected to PA.00.
     - Connect PA4 (DAC Channel1) and PA5 (DAC Channel2) pins to an oscilloscope.

In [ ]:
uint8_t Escalator8bit[6] = {0x0, 0x33, 0x66, 0x99, 0xCC, 0xFF};
uint16_t Sine12bit[32] = {
                      2047, 2447, 2831, 3185, 3498, 3750, 3939, 4056, 4095, 4056,
                      3939, 3750, 3495, 3185, 2831, 2447, 2047, 1647, 1263, 909, 
                      599, 344, 155, 38, 0, 38, 155, 344, 599, 909, 1263, 1647};

const uint32_t ptr_remote_RAM_SinWave = ptr_remote_RAM + sizeof(Escalator8bit);

Upload the Escalator wave data to MCU memory using ***remcu_store2mem*** function(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):
![Image](img/escalator_mem.png)

In [ ]:
remcu_store2mem(ptr_remote_RAM, Escalator8bit, sizeof(Escalator8bit))

Upload the Sine wave data to MCU memory using ***remcu_store2mem*** function(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):
![Image](img/sine_mem.png)
>Note that Community and Education version of the  [**REMCU**](https://remotemcu.github.io/) library has a ***32-bytes limit*** on the memory operation. Therefore, if you need to copy big data, you can do it in parts.  
Also the Community and Education versions have restrictions on memory region where you store and load data. You can use first 8 KBytes of first memory bank(STM32F407 chip). It is from 0x20000000 to (0x20000000 + 8*1024) address. To clarify that, see the [Download](https://remotemcu.github.io/download) page of your chip or the START_AVAILABLE_MEMORY_REGION and EMD_AVAILABLE_MEMORY_REGION constants in device_defines.h(is in remcu_include folder)

In [ ]:
remcu_store2mem(ptr_remote_RAM_SinWave, (uint8_t*)Sine12bit, 32);          //copy first half of data
remcu_store2mem(ptr_remote_RAM_SinWave + 32, (uint8_t*)Sine12bit + 32, 32) //copy second half of data

TIM6 Configuration:

In [ ]:
/* DMA1 clock and GPIOA clock enable (to be used with DAC) */
  RCC_AHB1PeriphClockCmd(RCC_AHB1Periph_DMA1 | RCC_AHB1Periph_GPIOA, ENABLE);

  /* DAC Periph clock enable */
  RCC_APB1PeriphClockCmd(RCC_APB1Periph_DAC, ENABLE);

  /* DAC channel 1 & 2 (DAC_OUT1 = PA.4)(DAC_OUT2 = PA.5) configuration */
  GPIO_InitStructure.GPIO_Pin = GPIO_Pin_4 | GPIO_Pin_5;
  GPIO_InitStructure.GPIO_Mode = GPIO_Mode_AN;
  GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL;
  GPIO_Init(GPIOA, &GPIO_InitStructure);

  /* TIM6 Periph clock enable */
  RCC_APB1PeriphClockCmd(RCC_APB1Periph_TIM6, ENABLE);

  /* Time base configuration */
  TIM_TimeBaseStructInit(&TIM_TimeBaseStructure);
  TIM_TimeBaseStructure.TIM_Period = 0xFF;
  TIM_TimeBaseStructure.TIM_Prescaler = 0;
  TIM_TimeBaseStructure.TIM_ClockDivision = 0;
  TIM_TimeBaseStructure.TIM_CounterMode = TIM_CounterMode_Up; 
  TIM_TimeBaseInit(TIM6, &TIM_TimeBaseStructure);

  /* TIM6 TRGO selection */
  TIM_SelectOutputTrigger(TIM6, TIM_TRGOSource_Update);
  
  /* TIM6 enable counter */
  TIM_Cmd(TIM6, ENABLE);

DAC and DMA configuration:

In [ ]:
  #define DAC_DHR12R2_ADDRESS    0x40007414
  #define DAC_DHR8R1_ADDRESS     0x40007410

  DAC_InitTypeDef  DAC_InitStructure;

  DAC_DeInit();

  /*-------------------- DAC Channel1 Escalator Configuration --------------------*/
  DAC_InitStructure.DAC_Trigger = DAC_Trigger_T6_TRGO;
  DAC_InitStructure.DAC_WaveGeneration = DAC_WaveGeneration_None;
  DAC_InitStructure.DAC_OutputBuffer = DAC_OutputBuffer_Enable;
  DAC_Init(DAC_Channel_1, &DAC_InitStructure);

  /* DMA1_Stream6 channel7 configuration **************************************/
  DMA_DeInit(DMA1_Stream6);
  DMA_InitStructure.DMA_Channel = DMA_Channel_7;  
  DMA_InitStructure.DMA_PeripheralBaseAddr = DAC_DHR8R1_ADDRESS;
  DMA_InitStructure.DMA_Memory0BaseAddr = ptr_remote_RAM;
  DMA_InitStructure.DMA_BufferSize = 6; //sizeof(Escalator8bit)
  DMA_InitStructure.DMA_PeripheralDataSize = DMA_PeripheralDataSize_Byte;
  DMA_InitStructure.DMA_MemoryDataSize = DMA_MemoryDataSize_Byte;
  DMA_InitStructure.DMA_DIR = DMA_DIR_MemoryToPeripheral;
  DMA_InitStructure.DMA_PeripheralInc = DMA_PeripheralInc_Disable;
  DMA_InitStructure.DMA_MemoryInc = DMA_MemoryInc_Enable;
  DMA_InitStructure.DMA_Mode = DMA_Mode_Circular;
  DMA_InitStructure.DMA_Priority = DMA_Priority_High;
  DMA_InitStructure.DMA_FIFOMode = DMA_FIFOMode_Disable;         
  DMA_InitStructure.DMA_FIFOThreshold = DMA_FIFOThreshold_HalfFull;
  DMA_InitStructure.DMA_MemoryBurst = DMA_MemoryBurst_Single;
  DMA_InitStructure.DMA_PeripheralBurst = DMA_PeripheralBurst_Single;
  DMA_Init(DMA1_Stream6, &DMA_InitStructure);    

  /* Enable DMA1_Stream6 */
  DMA_Cmd(DMA1_Stream6, ENABLE);
  
  /* Enable DAC Channel1 */
  DAC_Cmd(DAC_Channel_1, ENABLE);

  /* Enable DMA for DAC Channel1 */
  DAC_DMACmd(DAC_Channel_1, ENABLE);

//-------------------- DAC  Channel2 SineWave Configuration --------------------------------//
  /* DAC channel2 Configuration */
  DAC_InitStructure.DAC_Trigger = DAC_Trigger_T6_TRGO;
  DAC_InitStructure.DAC_WaveGeneration = DAC_WaveGeneration_None;
  DAC_InitStructure.DAC_OutputBuffer = DAC_OutputBuffer_Enable;
  DAC_Init(DAC_Channel_2, &DAC_InitStructure);

  /* DMA1_Stream5 channel7 configuration **************************************/
  DMA_DeInit(DMA1_Stream5);
  DMA_InitStructure.DMA_Channel = DMA_Channel_7;  
  DMA_InitStructure.DMA_PeripheralBaseAddr = (uint32_t)DAC_DHR12R2_ADDRESS;
  DMA_InitStructure.DMA_Memory0BaseAddr = (uint32_t)ptr_remote_RAM_SinWave;
  DMA_InitStructure.DMA_DIR = DMA_DIR_MemoryToPeripheral;
  DMA_InitStructure.DMA_BufferSize = 32; //sizeof(Sine12bit)
  DMA_InitStructure.DMA_PeripheralInc = DMA_PeripheralInc_Disable;
  DMA_InitStructure.DMA_MemoryInc = DMA_MemoryInc_Enable;
  DMA_InitStructure.DMA_PeripheralDataSize = DMA_PeripheralDataSize_HalfWord;
  DMA_InitStructure.DMA_MemoryDataSize = DMA_MemoryDataSize_HalfWord;
  DMA_InitStructure.DMA_Mode = DMA_Mode_Circular;
  DMA_InitStructure.DMA_Priority = DMA_Priority_High;
  DMA_InitStructure.DMA_FIFOMode = DMA_FIFOMode_Disable;
  DMA_InitStructure.DMA_FIFOThreshold = DMA_FIFOThreshold_HalfFull;
  DMA_InitStructure.DMA_MemoryBurst = DMA_MemoryBurst_Single;
  DMA_InitStructure.DMA_PeripheralBurst = DMA_PeripheralBurst_Single;
  DMA_Init(DMA1_Stream5, &DMA_InitStructure);

  /* Enable DMA1_Stream5 */
  DMA_Cmd(DMA1_Stream5, ENABLE);

  /* Enable DAC Channel2 */
  DAC_Cmd(DAC_Channel_2, ENABLE);

  /* Enable DMA for DAC Channel2 */
  DAC_DMACmd(DAC_Channel_2, ENABLE);

<details>
  <summary><b>Result (click to show)</b></summary>

![Image](img/dac1.png)
</details>

You can remotely change the MCU memory using the **remcu_store2mem** function and the DAC will play the modified content.
>Note that Community and Education version of the  [**REMCU**](https://remotemcu.github.io/) library has six bytes of minimum size limit on the memory operation. Therefore, you can not copy less six bytes at a time.

In [ ]:
Sine12bit[9] = 0;
remcu_store2mem(ptr_remote_RAM_SinWave, (uint8_t*)Sine12bit, 32)

<details>
  <summary><b>Result (click to show)</b></summary>

![Image](img/dac2.png)
</details>

### Extra Utility Function

The ***ReMCU_getVersion***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function returns a pointer to a char array containing information about the REMCU lib versions

In [ ]:
remcu_getVersion()

The ***ReMCU_disconnect***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function disconnects from the debug server. It is finalizing work with REMCU Library.

In [ ]:
remcu_disconnect()